## Import

In [4]:
import pandas as pd
import re
from matplotlib import pyplot
import seaborn as sns
import numpy as np

np.random.seed(843923)

## Load File


In [6]:
tweet_feed = pd.read_csv('Dataset/cleanprep.csv', delimiter=',', error_bad_lines=False,engine = 'python', header = None)

<ipython-input-6-3b25e9967bd0>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  tweet_feed = pd.read_csv('Dataset/cleanprep.csv', delimiter=',', error_bad_lines=False,engine = 'python', header = None)


In [7]:
tweet_feed.head()

,0,1,2,3,4
0,2018-07-11 19:35:15.363270,b'tj',"b""Next two weeks prob v boring (climb up to 9k...",0.007273,0.590909
1,2018-07-11 19:35:15.736769,b'Kool_Kheart',b'@Miss_rinola But you\xe2\x80\x99ve heard abo...,0.000000,0.000000
2,2018-07-11 19:35:15.744769,b'Gary Lang',b'Duplicate skilled traders automatically with...,0.625000,0.500000
3,2018-07-11 19:35:15.867339,b'Jobs in Fintech',b'Project Manager - Technical - FinTech - Cent...,0.000000,0.175000
4,2018-07-11 19:35:16.021448,b'ERC20',"b'Coinbase App Downloads Drop, Crypto Hype Fad...",0.333333,0.500000


## Preprocessing

In [8]:
tweet_feed.columns =['dt', 'name','text','polarity','sensitivity']

In [9]:
tweet_feed

,dt,name,text,polarity,sensitivity
0,2018-07-11 19:35:15.363270,b'tj',"b""Next two weeks prob v boring (climb up to 9k...",0.007273,0.590909
1,2018-07-11 19:35:15.736769,b'Kool_Kheart',b'@Miss_rinola But you\xe2\x80\x99ve heard abo...,0.000000,0.000000
2,2018-07-11 19:35:15.744769,b'Gary Lang',b'Duplicate skilled traders automatically with...,0.625000,0.500000
3,2018-07-11 19:35:15.867339,b'Jobs in Fintech',b'Project Manager - Technical - FinTech - Cent...,0.000000,0.175000
4,2018-07-11 19:35:16.021448,b'ERC20',"b'Coinbase App Downloads Drop, Crypto Hype Fad...",0.333333,0.500000
...,...,...,...,...,...
1412996,2018-07-24 08:17:01.654579,b'duong dao',b'#BlockBuratino https://t.co/bdlyjnIqZC',0.000000,0.000000
1412997,2018-07-24 08:17:02.342093,b'Ali Mohd',b'Cointelegraph: #CT_index Provided by Cointel...,0.000000,0.000000
1412998,2018-07-24 08:17:02.951448,b'Gilles Cottyn',b'Congrats @Bitcoin for reaching $ 8k again \x...,0.000000,0.000000
1412999,2018-07-24 08:17:03.388879,b'Mr. Jackson \xe2\x98\xaf',b'BTC wants to breakout again this morning and...,0.200000,0.100000


In [11]:
tweet_feed =tweet_feed.drop(['name','text'], axis=1)

In [12]:
tweet_feed.head()

,dt,polarity,sensitivity
0,2018-07-11 19:35:15.363270,0.007273,0.590909
1,2018-07-11 19:35:15.736769,0.000000,0.000000
2,2018-07-11 19:35:15.744769,0.625000,0.500000
3,2018-07-11 19:35:15.867339,0.000000,0.175000
4,2018-07-11 19:35:16.021448,0.333333,0.500000


In [13]:
tweet_feed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413001 entries, 0 to 1413000
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   dt           1413001 non-null  object 
 1   polarity     1413001 non-null  float64
 2   sensitivity  1413001 non-null  float64
dtypes: float64(2), object(1)
memory usage: 32.3+ MB


In [14]:
tweet_feed['dt'] = pd.to_datetime(tweet_feed['dt'])

In [15]:
tweet_feed['DateTime'] = tweet_feed['dt'].dt.floor('h')
tweet_feed.head()

,dt,polarity,sensitivity,DateTime
0,2018-07-11 19:35:15.363270,0.007273,0.590909,2018-07-11 19:00:00
1,2018-07-11 19:35:15.736769,0.000000,0.000000,2018-07-11 19:00:00
2,2018-07-11 19:35:15.744769,0.625000,0.500000,2018-07-11 19:00:00
3,2018-07-11 19:35:15.867339,0.000000,0.175000,2018-07-11 19:00:00
4,2018-07-11 19:35:16.021448,0.333333,0.500000,2018-07-11 19:00:00


In [16]:
tweet_feed_hour = tweet_feed.groupby(pd.Grouper(key='dt' ,freq='H')).size().reset_index(name='tweet_vol')

In [17]:
tweet_feed_hour.head()

,dt,tweet_vol
0,2018-07-11 19:00:00,1747
1,2018-07-11 20:00:00,4354
2,2018-07-11 21:00:00,4432
3,2018-07-11 22:00:00,3980
4,2018-07-11 23:00:00,3830


In [18]:
tweet_feed_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dt         302 non-null    datetime64[ns]
 1   tweet_vol  302 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.8 KB


In [19]:
tweet_feed_hour.index = pd.to_datetime(tweet_feed_hour.index)
tweet_feed_hour = tweet_feed_hour.set_index('dt')

In [20]:
tweet_feed_hour.head()

,tweet_vol
dt,
2018-07-11 19:00:00,1747
2018-07-11 20:00:00,4354
2018-07-11 21:00:00,4432
2018-07-11 22:00:00,3980
2018-07-11 23:00:00,3830


In [21]:
tweet_feed_hour.index = pd.to_datetime(tweet_feed_hour.index)

In [22]:
tweet_feed_hour['tweet_vol'] = tweet_feed_hour['tweet_vol'].astype(float)

In [23]:
tweet_df = tweet_feed.groupby('DateTime').agg(lambda  x: x.mean())

In [24]:
tweet_df['Tweet_vol'] = tweet_feed_hour['tweet_vol']

In [25]:
tweet_df = tweet_df.drop(tweet_df.index[0])
tweet_df.head()

,dt,polarity,sensitivity,Tweet_vol
DateTime,,,,
2018-07-11 20:00:00,2018-07-11 20:27:49.510636288,0.102657,0.216148,4354.0
2018-07-11 21:00:00,2018-07-11 21:28:35.636368640,0.098004,0.218612,4432.0
2018-07-11 22:00:00,2018-07-11 22:27:44.646705152,0.096688,0.231342,3980.0
2018-07-11 23:00:00,2018-07-11 23:28:06.455850496,0.103997,0.217739,3830.0
2018-07-12 00:00:00,2018-07-12 00:28:47.975385344,0.094383,0.195256,3998.0


In [26]:
tweet_df = tweet_df.drop(['dt'], axis=1)

In [27]:
tweet_df.tail()

,polarity,sensitivity,Tweet_vol
DateTime,,,
2018-07-24 04:00:00,0.121358,0.236000,4475.0
2018-07-24 05:00:00,0.095163,0.216924,4808.0
2018-07-24 06:00:00,0.088992,0.220173,6036.0
2018-07-24 07:00:00,0.091439,0.198279,6047.0
2018-07-24 08:00:00,0.071268,0.218217,2444.0


In [30]:
btc_pricing = pd.read_csv('Dataset/btcSave2.csv', error_bad_lines=False, engine= 'python')

<ipython-input-30-950e4a96d7e9>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  btc_pricing = pd.read_csv('Dataset/btcSave2.csv', error_bad_lines=False, engine= 'python')


In [31]:
btc_pricing['Timestamp'] = pd.to_datetime(btc_pricing['Timestamp'])
btc_pricing= btc_pricing.set_index(pd.DatetimeIndex(btc_pricing['Timestamp']))

In [32]:
btc_pricing.head()

,Timestamp,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Timestamp,,,,,,,,
2018-07-10 01:00:00,2018-07-10 01:00:00,6666.75,6683.90,6635.59,6669.73,281.73,1875693.72,6657.70
2018-07-10 02:00:00,2018-07-10 02:00:00,6662.44,6674.60,6647.00,6647.00,174.10,1160103.29,6663.38
2018-07-10 03:00:00,2018-07-10 03:00:00,6652.52,6662.82,6621.99,6632.53,231.41,1536936.22,6641.70
2018-07-10 04:00:00,2018-07-10 04:00:00,6631.17,6655.48,6625.54,6635.92,120.38,799154.77,6638.52
2018-07-10 05:00:00,2018-07-10 05:00:00,6632.81,6651.06,6627.64,6640.57,94.00,624289.31,6641.32


In [33]:
btc_pricing.drop(['Timestamp'], axis=1)

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Timestamp,,,,,,,
2018-07-10 01:00:00,6666.75,6683.90,6635.59,6669.73,281.73,1875693.72,6657.70
2018-07-10 02:00:00,6662.44,6674.60,6647.00,6647.00,174.10,1160103.29,6663.38
2018-07-10 03:00:00,6652.52,6662.82,6621.99,6632.53,231.41,1536936.22,6641.70
2018-07-10 04:00:00,6631.17,6655.48,6625.54,6635.92,120.38,799154.77,6638.52
2018-07-10 05:00:00,6632.81,6651.06,6627.64,6640.57,94.00,624289.31,6641.32
...,...,...,...,...,...,...,...
2018-07-23 21:00:00,7746.99,7763.59,7690.16,7706.00,237.63,1836633.86,7729.06
2018-07-23 22:00:00,7699.13,7759.99,7690.50,7750.09,63.31,489000.25,7724.30
2018-07-23 23:00:00,7754.57,7777.00,7715.45,7722.32,280.46,2173424.81,7749.63


In [34]:
Final_btc_pricing = pd.merge(tweet_df, btc_pricing, how='inner', left_index=True, right_index=True)

In [35]:
Final_btc_pricing.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 294 entries, 2018-07-11 20:00:00 to 2018-07-24 01:00:00
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   polarity           294 non-null    float64       
 1   sensitivity        294 non-null    float64       
 2   Tweet_vol          294 non-null    float64       
 3   Timestamp          294 non-null    datetime64[ns]
 4   Open               294 non-null    float64       
 5   High               294 non-null    float64       
 6   Low                294 non-null    float64       
 7   Close              294 non-null    float64       
 8   Volume (BTC)       294 non-null    float64       
 9   Volume (Currency)  294 non-null    float64       
 10  Weighted Price     294 non-null    float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 27.6 KB
